<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Shadow-Master" data-toc-modified-id="Shadow-Master-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Shadow Master</a></span></li><li><span><a href="#Orbiting" data-toc-modified-id="Orbiting-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Orbiting</a></span></li><li><span><a href="#Solar-spectrum" data-toc-modified-id="Solar-spectrum-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Solar spectrum</a></span></li></ul></div>

# Shadow Master

In [1]:
shadow_in_pixels = 9.2
res = 0.5  # m
shadow_length = shadow_in_pixels * res
shadow_length

4.6

In [2]:
sun_elev = 26  # degree above horizon

In [1]:
from math import radians, tan

In [5]:
height = tan(radians(sun_elev)) * shadow_length
height

2.243569907402962

# Orbiting

Combining slew rate with surface travel rates etc.

In [1]:
slew_rate = 10  # deg/sec

Formula for velocity is $$v=\sqrt{\frac{G\cdot M_{body}}{R_{body}}}$$

In [1]:
from planetpy import constants as planconst

/Users/klay6683/Dropbox/src/planetpy/planetpy/factsheet_parse.py:46: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  df = df.convert_objects(convert_numeric=True)


In [10]:
from astropy import constants as const
import numpy as np
import math
from planetpy.constants import mars, earth
from astropy import units as u

class Orbiter:
    """Base class for orbiting calculations.
    
    Parameters
    ----------
    M : astropy.units.[mass]
        Mass of central body
    R : astropy.units.[length]
        Radius of circular orbit around body with mass M
        
    Attributes
    ----------
    G : astropy.constansts.G
        Gravitational constant
    """
    G = const.G
    
    def __init__(self, M, R):
        self.M = M
        self.R = R
        self.alt = R - self.R_body
        
    @property
    def v(self):
        "Return orbital velocity."
        return np.sqrt(self.G*self.M/self.R).decompose()
    
    @property
    def surface_circumference(self):
        "Calculate circumference of circular central body."
        return math.tau * self.R_body
    
    @property
    def orbitpath(self):
        "Return circumference of circular orbit around central body."
        return math.tau * self.R
    
    @property
    def T(self):
        "Return orbital period time T."
        return (self.orbitpath / self.v).decompose()
    
    @property
    def v_surf(self):
        "Return surface footprint speed."
        return (self.surface_circumference / self.T).decompose()
    
    def ground_travel(self, t):
        "Return footprint travel distance in time `t`."
        return self.v_surf * t
    
    @property
    def slew_rate(self):
        "Return calculated slew rate in degrees/second for targeting one ground spot."
        gt = self.ground_travel(1 * u.s)
        return np.arctan(gt/self.alt).to(u.degree) / u.s
    

class MarsOrbiter(Orbiter):
    """Specialized Orbiter class for Mars.
    
    Parameters
    ----------
    alt : astropy.unit.length[km, m, etc]
        Value for Orbital altitude above ground.
        
    Attributes
    ----------
    M : astropy.units.mass
        Mass of Mars
    R_body : astropy.units.length
        Radius of Mars
    """
    M = mars.mass * 1e24 * u.kg
    R_body = mars.diameter / 2 * u.km
    
    def __init__(self, alt):
        super().__init__(self.M, self.R_body+alt)
        
        
class EarthOrbiter(Orbiter):
    
    M = earth.mass * 1e24 * u.kg
    R_body = earth.diameter / 2 * u.km
    
    def __init__(self, d):
        super().__init__(self.M, self.R_body+d)

In [11]:
orb = MarsOrbiter(350 * u.km)

In [12]:
orb.T.to(u.hour)

<Quantity 1.93315114 h>

In [13]:
orb.slew_rate

<Quantity 0.50190638 deg / s>

In [22]:
gt = orb.ground_travel(10 * u.millisecond) + 0 *u.meter
np.arctan(gt/orb.alt).to(u.degree)

<Quantity 0.00501919 deg>

In [23]:
gt = orb.ground_travel(10 * u.millisecond) + 5 *u.meter
np.arctan(gt/orb.alt).to(u.degree)

<Quantity 0.0058377 deg>

In [26]:
(0.584 - 0.502)

0.08199999999999996

In [15]:
orb.alt

<Quantity 350. km>

In [16]:
orb.v

<Quantity 3382.04460853 m / s>

In [14]:
orb.v_surf

<Quantity 3005.67205843 m / s>

In [50]:
from planetpy.missions import planning

In [61]:
orb = planning.MarsOrbiter(250)

Assuming kilometers as unit for input parameter.


# Solar spectrum

In [17]:
%matplotlib ipympl

In [216]:
from pyspectral.solar import (SolarIrradianceSpectrum, 
                              TOTAL_IRRADIANCE_SPECTRUM_2000ASTM,
                             )
import astropy.units as u
from astropy.constants import h, c
from scipy.interpolate import InterpolatedUnivariateSpline
import math

class Radiometry:
    E_w_unit_in = u.Watt/u.m/u.m/u.micron
    E_w_unit_out = u.Watt/u.m/u.m/u.nm
    E_ph_unit = 1/(u.s*u.m*u.m*u.nm)
    lw = 0.75
    rootpath = Path("/Users/klay6683/Documents/proposals/2018/MAPSE/")
    def __init__(self, wave1=200*u.nm, wave2=1200*u.nm, dlambda=1*u.nm,
                 i=75*u.deg, d=1.51):
        self.wave1 = wave1
        self.wave2 = wave2
        self.dlambda = dlambda
        self.i = i  # incidence angle
        self.d = d  # Mars distance in AU (scaling the solar flux)
        sol = SolarIrradianceSpectrum(TOTAL_IRRADIANCE_SPECTRUM_2000ASTM)
        sol.interpolate(dlambda=dlambda.to(u.micron).value, 
                        ival_wavelength=(wave1.to(u.micron).value,
                                         wave2.to(u.micron).value))
        self.waves = (sol.ipol_wavelength*u.micron).to(u.nm)
        self.E_w = (sol.ipol_irradiance*self.E_w_unit_in).to(self.E_w_unit_out)
        
        self.reflectance = pd.read_csv(self.rootpath / 'giza_crism_bright_areas.csv')
        self.reflectance = self.reflectance.sort_values(by='Wavelength[nm]')
        
        self.QE = pd.read_csv(self.rootpath / 'midband_coated_QE.csv')
        self.QE = self.QE.sort_values(by='Wavelength[nm]')
        
    @property
    def QE_rsr(self):
        d = {}
        d['wavelength'] = self.QE.iloc[:, 0]
        d['response'] = self.QE.iloc[:, 1]
        return d
        
    @property
    def rsr(self):
        d = {}
        d['wavelength'] = self.reflectance.iloc[:, 0]
        d['response'] = self.reflectance.iloc[:, 1]
        return d
    
    def plot_E_w(self, ax=None):
        xlim = [self.wave1.value, self.wave2.value]
        
        if ax is None:
            _, ax = plt.subplots(figsize=(8,4))

        ax.plot(self.waves, self.E_w, lw=self.lw)
        ax.set_xlim(xlim)
        ax.set_ylim(0, 2.5)
        ax.grid(True)
        ax.set_xlabel(f"Wavelength [{self.waves.unit}]")
        ax.set_ylabel(f"Spectral irradiance [{self.E_w.unit}]")
        ax.set_title("Spectral irradiance ($E_w$)")
        
    @property
    def ph_per_energy(self):
        return self.waves/(h*c)
    
    @property
    def E_ph(self):
        return (self.E_w * self.ph_per_energy).to(self.E_ph_unit)
    
    def plot_E_ph(self, ax=None):
        xlim = [self.wave1.value, self.wave2.value]
        
        if ax is None:
            _, ax = plt.subplots(figsize=(8,4))

        ax.plot(self.waves, self.E_ph, lw=self.lw)
        ax.set_xlim(xlim)
        ax.set_ylim(ymin=0, ymax=6e18)
        ax.grid(True)
        ax.set_xlabel(f"Wavelength [{self.waves.unit}]")
        ax.set_ylabel(f"Spectral irradiance [{self.E_ph.unit}]")
        ax.set_title("Spectral irradiance ($E_{ph}$)")
        
            
    @property
    def resp_ipol(self):
        ius = InterpolatedUnivariateSpline(self.rsr['wavelength'],
                                           self.rsr['response'],
                                           k=1)
        return ius(self.waves.value)
    
    @property
    def QE_ipol(self):
        ius = InterpolatedUnivariateSpline(self.QE_rsr['wavelength'],
                                           self.QE_rsr['response'],
                                           k=1)
        return ius(self.waves.values)
        
    @property
    def L_surf(self):
        term1 = self.E_ph/self.d**2
        term2 = math.cos(self.i.to(u.rad).value) / math.pi
        term3 = self.resp_ipol
        return term1*term2*term3

In [228]:
radio = Radiometry()

In [229]:
radio.QE_ipol

AttributeError: 'Quantity' object has no 'values' member

In [232]:
fig, ax = plt.subplots(ncols=2, figsize=(10,4))
radio.plot_E_w(ax[0])
radio.plot_E_ph(ax[1])

/Users/klay6683/miniconda3/envs/stable/lib/python3.6/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [231]:
plt.close('all')

In [233]:
from scipy.interpolate import InterpolatedUnivariateSpline


In [234]:
plt.figure()
plt.plot(radio.rsr['wavelength'],
         radio.rsr['response'])

/Users/klay6683/miniconda3/envs/stable/lib/python3.6/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [236]:
plt.figure()
plt.plot(radio.QE_rsr['wavelength'],
         radio.QE_rsr['response'])

/Users/klay6683/miniconda3/envs/stable/lib/python3.6/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [237]:
ius_qe = InterpolatedUnivariateSpline(radio.QE_rsr['wavelength'], radio.QE_rsr['response'],
                                   k=1)

QE_ipol = ius(radio.waves.value)

In [238]:
ius = InterpolatedUnivariateSpline(radio.rsr['wavelength'], radio.rsr['response'],
                                   k=1)

resp_ipol = ius(radio.waves.value)

In [239]:
plt.figure()
plt.plot(radio.QE_rsr['wavelength'],
         radio.QE_rsr['response'])
plt.plot(radio.waves,
         QE_ipol, '--', color='red')

/Users/klay6683/miniconda3/envs/stable/lib/python3.6/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [226]:
plt.figure()
plt.plot(radio.rsr['wavelength'],
         radio.rsr['response'])
plt.plot(radio.waves,
         resp_ipol, '--', color='red')

/Users/klay6683/miniconda3/envs/stable/lib/python3.6/site-packages/matplotlib/__init__.py:932: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [160]:
InterpolatedUnivariateSpline?